# Abgabe 02 - Team 08

Importieren der verwendeten Bibliotheken:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

Tabellen laden:

In [2]:
opt = pd.read_csv('order_products__train.csv') # Current order
opp = pd.read_csv('order_products__prior.csv') # Old orders
op = pd.concat([opt, opp], ignore_index=True)
products = pd.read_csv('products.csv')
orders = pd.read_csv('orders.csv')

### Aufgabe1

Zu aller erst wollen wir einen Überblick über die Orders-Tabelle und deren zugehörigen Spalten bekommen:

In [3]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


Wir wollen eine Kopie der Orders-Tabelle als Hilfstabelle. Die Hilfstabelle soll den Durchschnitt der days_since_prior_order je Nutzer (user_id) anzeigen:

In [8]:
mean_days_since_prior_order = orders.groupby('user_id').mean()
mean_days_since_prior_order.head()

,order_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
user_id,,,,,
1,1.923450e+06,6.0,2.636364,10.090909,19.000000
2,1.768105e+06,8.0,2.066667,10.600000,16.285714
3,1.934104e+06,7.0,1.384615,16.307692,12.000000
4,1.507835e+06,3.5,4.500000,12.500000,17.000000
5,1.849579e+06,3.0,1.400000,15.000000,11.500000


Bevor wir die Durchschnittswerte der Hilfstabelle zu unserem ursprünglichen Dataframe (orders) hinzufügen können, müssen wir die betreffende Spalte distinct benennen und den Index resetten:

In [9]:
mean_days_since_prior_order.rename(columns = {'days_since_prior_order': 'mean_days_since_prior_order'}, inplace = True)

In [10]:
mean_days_since_prior_order = mean_days_since_prior_order.reset_index()
mean_days_since_prior_order.head()

,user_id,order_id,order_number,order_dow,order_hour_of_day,mean_days_since_prior_order
0,1,1.923450e+06,6.0,2.636364,10.090909,19.000000
1,2,1.768105e+06,8.0,2.066667,10.600000,16.285714
2,3,1.934104e+06,7.0,1.384615,16.307692,12.000000
3,4,1.507835e+06,3.5,4.500000,12.500000,17.000000
4,5,1.849579e+06,3.0,1.400000,15.000000,11.500000


Jetzt können wir die Spalte an den ursprünglichen Dataframe anfügen:

In [11]:
orders = pd.merge(mean_days_since_prior_order[['user_id','mean_days_since_prior_order']],orders)
orders.head()

,user_id,mean_days_since_prior_order,order_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,19.0,2539329,prior,1,2,8,NaN
1,1,19.0,2398795,prior,2,3,7,15.0
2,1,19.0,473747,prior,3,3,12,21.0
3,1,19.0,2254736,prior,4,4,7,29.0
4,1,19.0,431534,prior,5,4,15,28.0


Jetzt können wir die Spalte 'factor' berechnen und uns die ersten 20 Zeilen des Dataframes anzeigen lassen:

In [12]:
orders['factor'] = orders['days_since_prior_order']/orders['mean_days_since_prior_order']

In [13]:
orders.head(20)

,user_id,mean_days_since_prior_order,order_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,factor
0,1,19.000000,2539329,prior,1,2,8,NaN,NaN
1,1,19.000000,2398795,prior,2,3,7,15.0,0.789474
2,1,19.000000,473747,prior,3,3,12,21.0,1.105263
3,1,19.000000,2254736,prior,4,4,7,29.0,1.526316
4,1,19.000000,431534,prior,5,4,15,28.0,1.473684
5,1,19.000000,3367565,prior,6,2,7,19.0,1.000000
6,1,19.000000,550135,prior,7,1,9,20.0,1.052632
7,1,19.000000,3108588,prior,8,1,14,14.0,0.736842
8,1,19.000000,2295261,prior,9,1,16,0.0,0.000000
9,1,19.000000,2550362,prior,10,4,8,30.0,1.578947


### Aufgabe 2

Im ersten Schritt wollen wir einen Überblick über die Products-Tabelle bekommen:

In [14]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


Wir wollen uns anzeigen lassen, welche product_id Produkte mit dem Namen 'Organic Avocado' besitzten, um diese im weiteren Verlauf zu verwenden:

In [26]:
avocadoProductIDTabel = products[products['product_name'].str.contains('Organic Avocado', regex=False)]
avocadoProductIDTabel

,product_id,product_name,aisle_id,department_id
6217,6218,Organic Avocados,83,4
9188,9189,Organic Avocados 5 Count,32,4
25821,25822,Organic Avocado Oil,19,13
39000,39001,Organic Avocados Hass Variety,24,4
47765,47766,Organic Avocado,24,4


In [27]:
op.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


Da die Tabelle op nicht über die Wochentage verfügt, wird im nachfolgenden Schritt eine Hilfstabelle erstellt, die die Tabelle opp um die Wochentage ergänzt:

In [28]:
Avocadopart = pd.merge(orders[['order_id','order_dow']],
             op)
Avocadopart.head()

,order_id,order_dow,product_id,add_to_cart_order,reordered
0,2539329,2,196,1,0
1,2539329,2,14084,2,0
2,2539329,2,12427,3,0
3,2539329,2,26088,4,0
4,2539329,2,26405,5,0


Daraufhin erstellen wir eine Hilfstabelle, die uns für jeden Tag der Woche die Anzahl der Bestellungen angibt, sowie eine zweite Hilfstabelle, die uns für jeden Tag der Woche die Anzahl der Bestellungen angibt, die Organic Avocados beinhalten:

In [29]:
Avocadopart_2 = Avocadopart.pivot_table('order_id', 'order_dow', aggfunc = 'count')
Avocadopart_2.head()

,order_id
order_dow,
0,6533692
1,5871834
2,4378360
3,3998498
4,3942696


In [30]:
Avocadopart_2 = Avocadopart_2.reset_index()

In [44]:
Avocadopart_3 = Avocadopart[Avocadopart.product_id.isin(avocadoProductIDTabel.product_id.tolist())].pivot_table('order_id', 'order_dow', aggfunc ='count')
Avocadopart_3

,order_id
order_dow,
0,43146
1,34638
2,23286
3,20521
4,20001
5,22383
6,26986


In [45]:
Avocadopart_3.rename(columns = {'order_id': 'orders_containing_Avocado'}, inplace = True)

In [46]:
Avocadopart_3

,orders_containing_Avocado
order_dow,
0,43146
1,34638
2,23286
3,20521
4,20001
5,22383
6,26986


In [47]:
Avocadopart_3 = Avocadopart_3.reset_index()

Schlussendlich vereinen wir die dataframes in einem finalen Frame (Avocadopart_4) ud teilen die Bestellungen die Avocado enthalten durch die Gesamtzahl der Bestellungen, um den Anteil der Bestellungen pro Tag zu erhalten, die Avocados beinhalten:

In [48]:
Avocadopart_4 = pd.merge(Avocadopart_3[['order_dow','orders_containing_Avocado']],
             Avocadopart_2)
Avocadopart_4

,order_dow,orders_containing_Avocado,order_id
0,0,43146,6533692
1,1,34638,5871834
2,2,23286,4378360
3,3,20521,3998498
4,4,20001,3942696
5,5,22383,4386443
6,6,26986,4707583


In [49]:
Avocadopart_4['orders_containing_Avocado_in_perc'] = Avocadopart_4['orders_containing_Avocado']/Avocadopart_4['order_id']

In [50]:
Avocadopart_4

,order_dow,orders_containing_Avocado,order_id,orders_containing_Avocado_in_perc
0,0,43146,6533692,0.006604
1,1,34638,5871834,0.005899
2,2,23286,4378360,0.005318
3,3,20521,3998498,0.005132
4,4,20001,3942696,0.005073
5,5,22383,4386443,0.005103
6,6,26986,4707583,0.005732


### Aufgabe 3

Im ersten Schritt lassen wir uns die Anzahl an Bestellungen ausgeben, die Avocados enthalten.

In [52]:
# Find orders with avocados (Product 47766)
ordersWithAvocados = (op.loc[op['product_id'].isin(avocadoProductIDTabel.product_id.tolist()),'order_id'])

print(f"Anzahl an Bestellposten Avocados: {len(ordersWithAvocados)}")

Anzahl an Bestellposten Avocados: 190961


Wir benötigen für den weiteren Verlauf eine Spalte in unserer orders Tabelle, die angibt, ob eine Bestellung Avocado(s) enthalten hat oder nicht. Da wir nicht alle Spalten der orders Tabelle benötigen, werden die überflüssigen Spalten gedropped.

In [53]:
orders.drop(["eval_set", "order_dow", "order_hour_of_day", "days_since_prior_order", "mean_days_since_prior_order", "factor"], axis=1, inplace=True)

In [54]:
# Mark orders with avocados in orders table
orders.loc[:,'contains_avocados']=orders['order_id'].isin(ordersWithAvocados)
orders.head(5)

,user_id,order_id,order_number,contains_avocados
0,1,2539329,1,False
1,1,2398795,2,False
2,1,473747,3,False
3,1,2254736,4,False
4,1,431534,5,False


In [55]:
result = len ( orders [ orders['contains_avocados'] == True ])
print(f"Anzahl an gefundenen Bestellungen mit Avocados: {result}")

Anzahl an gefundenen Bestellungen mit Avocados: 190909


### Ansatz:
Um die Relation der vorherigen Bestellung (Enthält Avocados oder nicht) abzubilden wird ein Comparison frame erstellt.
Dieser beinhaltet als "order_number" auf welche Folgebestellung sich der Wahrheitswert "priorOrderContainsAvocados" bezieht.

In [56]:
# Create Comparison frame
orderComparison = orders.copy()
orderComparison.rename({'contains_avocados': 'priorOrderContainsAvocados'}, axis=1, inplace=True)
orderComparison["order_number"] = orderComparison["order_number"].apply(lambda row: row+1 )
orderComparison.head(5)

,user_id,order_id,order_number,priorOrderContainsAvocados
0,1,2539329,2,False
1,1,2398795,3,False
2,1,473747,4,False
3,1,2254736,5,False
4,1,431534,6,False


Die beiden Dataframes werden nun durch eine "Left Outer Join"-Operation über ihre UserID und OrderNumber verbunden.

In [29]:
# Join the data frames together
orderMap = orders.merge(orderComparison, how='left', left_on=["user_id", "order_number"], right_on=["user_id", "order_number"])
orderMap.head(5)

# Alternative: Function
# oph['priorOrderContainsAvocados'] = oph.apply(lambda row: False if row["order_number"] == 1 or oph.iloc[row.name-1]["contains_avocados"] == False else True, axis=1)

,user_id,order_id_x,order_number,contains_avocados,order_id_y,priorOrderContainsAvocados
0,1,2539329,1,False,NaN,NaN
1,1,2398795,2,False,2539329.0,False
2,1,473747,3,False,2398795.0,False
3,1,2254736,4,False,473747.0,False
4,1,431534,5,False,2254736.0,False


In [60]:
resultA = len ( orderMap [ orderMap['contains_avocados'] == True ])
resultB = len ( orderMap [ (orderMap['contains_avocados'] == True) & (orderMap['priorOrderContainsAvocados'] == False) ])
resultC = len ( orderMap [ (orderMap['contains_avocados'] == True) & (orderMap['priorOrderContainsAvocados'] == True) ])
total = len ( orderMap )

print( f"Anzahl an Bestellungen mit Avocados: {resultA}" )
print( f"-> Wahrscheinlichkeit: { round(resultA / total, 2)} \n" )

print("Avokados nicht direkt davor bestellt:")
print( f"-> Anzahl: {resultB}" )
print( f"-> Wahrscheinlichkeit: { round(resultB / total, 2)} \n" )

print("Avokados direkt davor bestellt:")
print( f"-> Anzahl: {resultC}" )
print( f"-> Wahrscheinlichkeit: { round(resultC / total, 2)} \n" )

Anzahl an Bestellungen mit Avocados: 190909
-> Wahrscheinlichkeit: 0.06 

Avokados nicht direkt davor bestellt:
-> Anzahl: 92947
-> Wahrscheinlichkeit: 0.03 

Avokados direkt davor bestellt:
-> Anzahl: 81873
-> Wahrscheinlichkeit: 0.02 

